In [1]:
import os
import sys
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf
from google.cloud import bigquery

In [2]:
sys.path.append(os.path.abspath(".."))

In [3]:
from utils.bq_queries.anime_anime_data_queries import anime_anime_pair_ranking_query

In [4]:
def load_big_query_data(query):
    client = bigquery.Client(project="anime-rec-dev")
    dataset_ref = client.dataset("processed_area")
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query, job_config=job_config)
    data = query_job.to_dataframe()
    return data

# Pair ranking data

In [5]:
val_data = load_big_query_data(anime_anime_pair_ranking_query('VAL'))

In [6]:
val_data.head()

,anchor_anime,rel_anime_1,rel_anime_2,label
0,9926,2034,6045,0
1,9926,2034,11757,0
2,9926,2034,243,0
3,9926,9941,11123,0
4,9926,9941,853,0


In [7]:
val_data.shape

(4368989, 4)

In [8]:
anime_data = load_big_query_data("SELECT anime_id, score FROM `anime-rec-dev.processed_area.anime`")

In [9]:
anime_data = anime_data.fillna(0.0)

In [10]:
anime_data.head()

,anime_id,score
0,7016,6.78
1,10507,6.96
2,4075,6.84
3,36444,6.81
4,6172,7.64


In [11]:
anime_data.shape

(9192, 2)

In [12]:
val_ds = (
    tf.data.Dataset.from_tensor_slices(
        (
            {
                'anchor_anime' : tf.cast(val_data['anchor_anime'], tf.string),
                'rel_anime_1' : tf.cast(val_data['rel_anime_1'], tf.string),
                'rel_anime_2' : tf.cast(val_data['rel_anime_2'], tf.string),
            },
            val_data['label']
        )
    )
)
val_ds = val_ds.batch(2048).cache()

## Models

In [13]:
class AnimeAverageRatingModel(tf.keras.Model):
    def __init__(self, anime_ids, anime_scores):
        super().__init__()
        
        self.ratings = tf.lookup.StaticHashTable(
                tf.lookup.KeyValueTensorInitializer(anime_ids, anime_scores),
                default_value=-1
        )
    
    def call(self, features):

        pred_ratings = self.ratings.lookup(features['rel_anime'])

        return pred_ratings

In [14]:
class AnimeAnimePairClassificationModel(tf.keras.Model):
    '''
        Classification model that trains the scoring model
        This model takes as input three anime_ids and a label
            anchor_anime : the initial anime
            rel_anime_1 : first anime to be scored
            rel_anime_2 : second anime to be score
            label : 1 if rel_anime_1 is more relevant to anchor_anime than rel_anime_2
                    0 else
        Model computes the two scores and return
        sigmoid(score1 - score2) as binary classification prediction
        
    '''
    def __init__(self, anime_scoring_model):
        super().__init__()

        self.anime_scoring_model = anime_scoring_model

    def call(self, data):

        pred_score_1 = self.anime_scoring_model({
            'anchor_anime' : data["anchor_anime"],
            'rel_anime' : data["rel_anime_1"]
        })

        pred_score_2 = self.anime_scoring_model({
            'anchor_anime' : data["anchor_anime"],
            'rel_anime' : data["rel_anime_2"]
        })

        classification_score = tf.math.sigmoid(pred_score_1 - pred_score_2)
        return classification_score

In [15]:
average_anime_scoring_model = AnimeAverageRatingModel(anime_data['anime_id'], anime_data['score'])

In [16]:
average_anime_scoring_model({'anchor_anime' : tf.constant(['7016']), 'rel_anime' : tf.constant(['7016'])})

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([6.78])>

In [17]:
anime_anime_pair_classification_model = AnimeAnimePairClassificationModel(average_anime_scoring_model)
anime_anime_pair_classification_model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[
            tf.keras.metrics.BinaryAccuracy(),
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.AUC()
        ]
    )

In [18]:
anime_anime_pair_classification_model(
    {
        'anchor_anime' : tf.constant(['7016']), 
        'rel_anime_1' : tf.constant(['7016']),
        'rel_anime_2' : tf.constant(['7016']),
    }
)

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.5])>

In [19]:
anime_anime_pair_classification_model.evaluate(val_ds)

2134/2134 [==============================] - 13s 6ms/step - loss: 0.7971 - binary_accuracy: 0.4986 - precision: 0.4975 - recall: 0.4973 - auc: 0.4971


[0.7970818281173706,
 0.49863433837890625,
 0.49750518798828125,
 0.49726349115371704,
 0.49713659286499023]